In [1]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 19.3 MB/s eta 0:00:0000:01


In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [1]:
import fitz  # PyMuPDF
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

# Set pandas display options for left justification
pd.set_option('display.colheader_justify', 'left')
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_colwidth', None)  # Prevents truncating text in console display

# Open the PDF
pdf_document = "manual.pdf"
pdf = fitz.open(pdf_document)

# Load NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Function to extract keywords, limiting to 5-10 keywords per page
def extract_keywords(text, num_keywords=(5, 10)):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.isalnum() and word.lower() not in stop_words]
    keyword_freq = Counter(filtered_words)
    most_common_keywords = keyword_freq.most_common(num_keywords[1])
    selected_keywords = most_common_keywords[:max(num_keywords[0], len(most_common_keywords))]
    return ", ".join([kw[0] for kw in selected_keywords])

# Extract text from each page
text_data = []
row_id = 1
for page_num in range(len(pdf)):
    page = pdf.load_page(page_num)
    text = page.get_text("text")
    text = text.replace('\n', ' ')  # Remove all new lines within paragraphs
    
    # Extract keywords for the page
    keywords = extract_keywords(text.strip(), (5, 10))
    
    # Append the text data for the page
    text_data.append({
        "RowID": row_id,
        "Page#": page_num + 1,
        "keywords": keywords,
        "Page_text": text.strip()
    })
    row_id += 1

# Convert to pandas DataFrame
df = pd.DataFrame(text_data)

# Remove any rows with NaN values in the 'Page_text' column
df = df.dropna(subset=['Page_text'])


# Display DataFrame to verify without the auto index column
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.colheader_justify', 'left', 'display.width', 2000, 'display.max_colwidth', 500):
    df = df.style.set_properties(**{'text-align': 'left'})
    print(df)

df

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,RowID,Page#,keywords,Page_text
0,1,1,"Barista, Instruction, Book, BES870XL",the Barista Express™ Instruction Book - BES870XL /A
1,2,2,"safety, espresso, machine, 2, Breville, using, first, BREVILLE, RECOMMENDS, SAFETY","2 BREVILLE RECOMMENDS SAFETY FIRST At Breville we are very safety conscious. We design and manufacture consumer products with the safety of you, our valued customer, foremost in mind. In addition we ask that you exercise a degree of care when using any electrical appliance and adhere to the following precautions. IMPORTANT SAFEGUARDS READ ALL INSTRUCTIONS BEFORE USE AND SAVE FOR FUTURE REFERENCE • Carefully read all instructions before operating and save for future reference. • Remove any packaging material and promotional labels or stickers before using the espresso machine for the first time. • To eliminate a choking hazard for young children, remove and safely discard the protective cover fitted to the power plug of the espresso machine. • Do not place the espresso machine near the edge of a bench or table during operation. Ensure the surface is level, clean and free of water and other substances. 2 Breville recommends safety first 9 Components 11 Functions 24 Care & Cleaning 29 Troubleshooting 35 Coffees to try 36 French Contents"
2,3,3,"machine, espresso, use, water, place, hot, touch, surfaces, recommend, using","3 • Do not place the espresso machine on or near a hot gas or electric burner, or where it could touch a heated oven. • Do not use the espresso machine on metal surfaces, for example, a sink drain board. • Always ensure the espresso machine is properly assembled before connecting to a power outlet and operating. • Do not attempt to operate the espresso machine by any method other than those described in this book. • The espresso machine is not intended to be operated by means of an external timer or separate remote control system. • We recommend using cold, filtered water. We do not recommend the use of de-mineralized or distilled water as this may affect the taste of the espresso and how the machine is designed to function. Do not use any other liquid. • Never use the espresso machine without water in the water tank. • Ensure the portafilter is firmly inserted and secured into the group head before using the machine. • Never remove the portafilter during the brewing operation as the machine is under pressure. • Do not touch hot surfaces. Use handles or knobs. Allow the espresso machine to cool down before moving or cleaning any parts. • Do not place anything other than cups for warming on top of the espresso machine."
3,4,4,"appliance, use, espresso, machine, cord, hopper, Keep, unattended, cleaning, provided","4 • Do not operate the grinder without the hopper lid in position. Keep fingers, hands, hair, clothing and utensils away from the hopper during operation. • Always switch off the espresso machine by pressing the POWER button to off and unplug if appliance is to be left unattended, if not in use, before cleaning, before attempting to move the appliance, disassembling, assembling and when storing the appliance. • Keep the espresso machine and accessories clean. Follow the cleaning instructions provided in this book. • Do not use espresso machine attachments other than those provided. The use of accessory attachments not recommended by Breville may result in fire, electric shock or injury to persons. IMPORTANT SAFEGUARDS FOR ALL ELECTRICAL APPLIANCES • Fully unwind cord before use. • To protect against electric shock, do not immerse cord, plug or appliance in water or any other liquid. • Do not let the cord hang over the edge of a bench or table, touch hot surfaces, or become knotted. • Do not leave the appliance unattended when in use. • This appliance is not intended for use by persons (including children) with reduced physical, sensory or mental capabilities, or lack of experience or knowledge, unless they have been given supervision or instruction concerning use of the appliance by a pe